记忆增强曲线

In [ ]:
%matplotlib inline
import sys
import os

# 添加父目录到路径
sys.path.insert(0, os.path.abspath('..'))

import numpy as np
import matplotlib.pyplot as plt
from become_human.memory import calculate_stability_curve


n_points = 100
t = np.linspace(0, 1, n_points)
curve = np.zeros((n_points, 2))

values = []
zero_crossed = False
zero_point = 0

for ti in t:
    vvv = calculate_stability_curve(ti)
    values.append(vvv)
    if vvv >= 0 and not zero_crossed:
        zero_crossed = True
        zero_point = ti
# Generate the Bezier curve


# Plotting the control points and the curve
plt.figure(figsize=(8, 6))
plt.plot(t, values, label=f'max_y={max(values)}')

plt.title(f'max y={max(values)}, zero point={zero_point}')
plt.xlabel('retrievability')
plt.ylabel('stability')
plt.legend()
plt.grid(True)
#plt.axis('equal')  # Ensure equal scaling on both axes
plt.show()


指数分布图

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm


# 查找系统中的中文字体
fonts = [f.name for f in fm.fontManager.ttflist]
chinese_fonts = [f for f in fonts if any(ch in f for ch in ['SimHei', 'Microsoft', 'PingFang', 'STSong', 'Arial Unicode MS'])]

#print("可用的中文字体:", chinese_fonts[:5])  # 显示前5个

# 如果找到了中文字体，则设置
if chinese_fonts:
    plt.rcParams['font.sans-serif'] = chinese_fonts  # 使用第一个找到的中文字体
else:
    # 备选方案：使用系统默认字体并忽略缺失字符
    plt.rcParams['axes.unicode_minus'] = False  # 解决负号 '-' 显示为方块的问题

plt.rcParams['axes.unicode_minus'] = False  # 正常显示负号


# 设置参数
lambdas = [0.4, 0.8, 1.0]  # 不同的 lambda 值
x = np.linspace(0, 8, 400)       # x 轴范围（印象深度）
sample_size = 20000              # 采样数量

plt.figure(figsize=(10, 5))

for lam in lambdas:
    # 用 scale = 1/lambda 生成样本
    samples = np.random.exponential(scale=1/lam, size=sample_size)
    # 只显示 0~8 范围内的值（避免极端大值拉宽坐标）
    samples = samples[samples <= 8]
    plt.hist(samples, bins=80, alpha=0.5, density=True, 
             label=f'λ = {lam} (mean = {1/lam:.1f})')
plt.title('采样直方图（20,000 次模拟）')
plt.xlabel('印象深度')
plt.ylabel('频率密度')
plt.legend()
plt.grid(True, linestyle='--', alpha=0.6)

plt.tight_layout()
plt.show()

记忆模拟

In [ ]:
%matplotlib widget

import sys
import os

# 添加父目录到路径
sys.path.insert(0, os.path.abspath('..'))

from datetime import timedelta, datetime, timezone

import matplotlib.pyplot as plt
import numpy as np

from become_human.time import datetime_to_seconds
from become_human.memory import tick_memory, recall_memory

stable_time = 50
base_datetime = datetime(2025, 12, 21, 8, tzinfo=timezone.utc)
difficulty = 1.0
recall_times = [33, 35, 36, 40, 100, 200, 500]
total_time = 1000

metadata = {
    'retrievability': 1.0,
    'stable_time': stable_time,
    'last_accessed_agent_timeseconds': datetime_to_seconds(base_datetime),
    'difficulty': difficulty
}


time_points = np.linspace(0, total_time, 10000)
retrievability_values = []
forgot_values = []

deleted = False
deleted_point = []
recall_count = 0

for t in time_points:
    if t > 0:
        new_datetime = base_datetime + timedelta(seconds=t)
        metadata.update(tick_memory(metadata, new_datetime))
        retrievability = metadata['retrievability']

        if metadata.get('forgot'):
            if not deleted:
                deleted = True
                deleted_point = [t, retrievability]
        else:
            if recall_count < len(recall_times) and t >= recall_times[recall_count]:
                recall_count += 1
                metadata.update(recall_memory(metadata, new_datetime, 1.0))

    retrievability_values.append(metadata['retrievability'])

plt.figure(figsize=(10, 5))
plt.plot(time_points, retrievability_values)
if deleted_point:
    plt.scatter(deleted_point[0], deleted_point[1], color='red', label='deleted')
plt.xlabel('Time (s)')
plt.ylabel('Retrievability')
plt.title(f'Retrievability over Time, recall count: {recall_count}')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
%matplotlib inline
import sys
import os

# 添加父目录到路径
sys.path.insert(0, os.path.abspath('..'))

from datetime import datetime, timezone, timedelta
import numpy as np
import matplotlib.pyplot as plt
from become_human.memory import calculate_memory_datetime_alpha

base_datetime = datetime(2025, 12, 21, tzinfo=timezone.utc)

n_points = 86400
t = np.linspace(0, 86400, n_points)
curve = np.zeros((n_points, 2))

values = []

for ti in t:
    new_datetime = base_datetime + timedelta(seconds=ti)
    vvv = calculate_memory_datetime_alpha(new_datetime)
    values.append(vvv)
# Generate the Bezier curve


# Plotting the control points and the curve
plt.figure(figsize=(8, 6))
plt.plot(t, values)

plt.title(f'memory in a day')
plt.xlabel('time(s)')
plt.ylabel('alpha')
plt.legend()
plt.grid(True)
#plt.axis('equal')  # Ensure equal scaling on both axes
plt.show()


In [ ]:
%matplotlib widget
import ipywidgets as widgets
from IPython.display import display
import numpy as np
import matplotlib.pyplot as plt
from langchain_community.embeddings import DashScopeEmbeddings

memory_textbox = widgets.Text(
    value="The quick brown fox jumps over the lazy dog.",
    description='Memory:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='80%')
)

query1_textbox = widgets.Text(
    value="What does the fox do?",
    description='Query 1:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='80%')
)

query1_retrievability_slider: widgets.FloatSlider = widgets.FloatSlider(
    value=0.5,
    min=0.0,
    max=1.0,
    step=0.01,
    description='Retrievability 1:',
    style={'description_width': 'initial'}
)

query1_weight_slider: widgets.FloatSlider = widgets.FloatSlider(
    value=0.5,
    min=0.0,
    max=1.0,
    step=0.01,
    description='Weight (sim) 1:',
    style={'description_width': 'initial'}
)

query2_textbox = widgets.Text(
    value="Is the dog lazy?",
    description='Query 2:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='80%')
)

query2_retrievability_slider: widgets.FloatSlider = widgets.FloatSlider(
    value=0.5,
    min=0.0,
    max=1.0,
    step=0.01,
    description='Retrievability 2:',
    style={'description_width': 'initial'}
)

query2_weight_slider: widgets.FloatSlider = widgets.FloatSlider(
    value=0.5,
    min=0.0,
    max=1.0,
    step=0.01,
    description='Weight (sim) 2:',
    style={'description_width': 'initial'}
)


# 创建计算按钮
compute_button: widgets.Button = widgets.Button(
    description='Generate Embeddings',
    button_style='primary'
)

# 创建输出区域
output: widgets.Output = widgets.Output()

def cosine_similarity(vec1, vec2):
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

# 初始化嵌入模型
embedding_model = DashScopeEmbeddings(model="text-embedding-v4")

def compute_similarity(memory: str, query1: str, query2: str):
    """
    计算记忆与查询之间的相似度
    """
    result1 = 0
    result2 = 0
    memory_embedding = embedding_model.embed_documents([memory])[0]
    if query1:
        search_embedding1 = embedding_model.embed_query(query1)
        result1 = cosine_similarity(search_embedding1, memory_embedding)
    if query2:
        search_embedding2 = embedding_model.embed_query(query2)
        result2 = cosine_similarity(search_embedding2, memory_embedding)
    return result1, result2

# 存储结果的全局变量
sim1, sim2 = 0.0, 0.0

bars = None
texts: list[plt.Text] = []
created_plt = False
def on_compute_button_clicked(b):
    global sim1, sim2, bars, texts, created_plt

    if created_plt:
        plt.close()
    else:
        created_plt = True

    with output:
        output.clear_output()
        try:

            sim1, sim2 = compute_similarity(
                memory_textbox.value,
                query1_textbox.value,
                query2_textbox.value
            )
            
            # 获取滑块值
            retr1 = query1_retrievability_slider.value
            w1 = query1_weight_slider.value

            retr2 = query2_retrievability_slider.value
            w2 = query2_weight_slider.value
            
            # 计算最终得分
            score1 = w1 * sim1 + (1 - w1) * retr1
            score2 = w2 * sim2 + (1 - w2) * retr2
            
            # 显示结果
            print(f"Similarity Query 1: {sim1:.4f}")
            print(f"Similarity Query 2: {sim2:.4f}")
            print(f"Final Score Query 1: {score1:.4f}")
            print(f"Final Score Query 2: {score2:.4f}")
            
            # 可视化结果
            plt.figure(figsize=(6, 4))
            bars = plt.bar(['Query 1', 'Query 2'], [score1, score2], 
                          color=['skyblue', 'salmon'])
            plt.ylim(0, 1)
            plt.ylabel('Final Score')
            plt.title('Interactive Retrieval Scoring')
            
            # 在柱状图上显示数值
            texts = []
            for i, bar in enumerate(bars):
                height = bar.get_height()
                texts.append(plt.text(bar.get_x() + bar.get_width()/2., height,
                        f'{height:.3f}', ha='center', va='bottom'))


            plt.show()
            
        except Exception as e:
            print(f"Error: {e}")

# 绑定按钮点击事件
compute_button.on_click(on_compute_button_clicked)

def on_slider_changed(i):
    global texts

    if bars is None:
        return

    if i == 0:
        retr = query1_retrievability_slider.value
        w = query1_weight_slider.value
        sim = sim1
    elif i == 1:
        retr = query2_retrievability_slider.value
        w = query2_weight_slider.value
        sim = sim2
    else:
        raise ValueError("Invalid slider index")

    score = w * sim + (1 - w) * retr

    # 更新柱状图
    bars[i].set_height(score)
    texts[i].set(y=score, text=f'{score:.3f}')

def on_slider1_changed(v):
    on_slider_changed(0)
def on_slider2_changed(v):
    on_slider_changed(1)

query1_retrievability_slider.observe(on_slider1_changed, names='value')
query1_weight_slider.observe(on_slider1_changed, names='value')
query2_retrievability_slider.observe(on_slider2_changed, names='value')
query2_weight_slider.observe(on_slider2_changed, names='value')

# 创建控件布局
controls_layout = widgets.VBox([
    memory_textbox,
    query1_textbox,
    query1_retrievability_slider,
    query1_weight_slider,
    query2_textbox,
    query2_retrievability_slider,
    query2_weight_slider,
    compute_button,
    output
])

# 显示控件
display(controls_layout)
plt.close()